## Imports

In [ ]:
#Commonted out the below lines as the packages are already installed in the environment

# !pip install timm
# !pip install nibabel
# !pip install nilearn
# !pip install light-the-torch && ltt install torch
#!pip install torchio

import tensorflow as tf
#for neuroimaging data
import torchio as tio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
import os
import time
import gc


In [ ]:
torch.cuda.empty_cache()
gc.collect()

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

In [ ]:
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {torch.cuda.get_device_name()} for inference' if torch.cuda.is_available() else 'Using CPU for inference')

In [ ]:
#Set seed for reproducibility
seed = 42
torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
# cudnn.benchmark = False
# cudnn.deterministic = True
# torch.backends.cudnn.deterministic = True



## Dataset 

In [ ]:
root_dir = 'Dataset'

# List all subdirectories in the root directory
subdirectories = [x[0] for x in os.walk(root_dir)]

dataset = []

# Iterate over the subdirectories
for subdir in subdirectories:
    # Check if the subdirectory contains 'anat' in its path
    if 'anat' in subdir:
        # List all files in the subdirectory
        files = os.listdir(subdir)
        # Iterate over the files
        for file in files:
            # Check if the file is a .nii.gz file
            if file.endswith('.nii.gz'):
                file_path = os.path.join(subdir, file)
                # Load the Nifti image using TorchIO
                subject = tio.Subject(
                    mri = tio.ScalarImage(file_path),
                    id = int(os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path)))).replace('sub-', ''))
                )
                dataset.append(subject)
       

# Print the number of images in the dataset
print('Number of Images in Dataset: ' + str(len(dataset)))

# print the path of the first image
print('\nFirst Image Path in Dataset: ')
print(dataset[0].mri.path)
print('ID:', dataset[0]['id'])

print('\nFirst Image Dimensions in Dataset: ')
print(dataset[0].spatial_shape)


## Import Labels

In [ ]:
# check if a file named participants.tsv exists in the each of the subfolder under the root directory
# If it exists, load the file and display the contents
df = pd.DataFrame()
for subdir in subdirectories:
    if os.path.exists(os.path.join(subdir, 'participants.csv')):
        print('Found participants.tsv in ' + subdir)
        new_df = pd.read_csv(os.path.join(subdir, 'participants.csv'), sep=',', header=0)
        new_df = new_df[['participant_id', 'age', 'gender', 'dx']]
        df = pd.concat([df, new_df], ignore_index=True)

# Print the resulting DataFrame
df.head()


In [ ]:
# Assign each row of the df to the corresponding subject in the dataset
for subject in dataset:
    subject_id = subject['id']
    if subject_id not in df['participant_id'].values:
        print(subject_id + ' dataframe does not have this subject_id')
    subject['age'] = df.loc[df['participant_id'] == subject_id, 'age'].values[0]
    subject['gender'] = df.loc[df['participant_id'] == subject_id, 'gender'].values[0]
    subject['dx'] = df.loc[df['participant_id'] == subject_id, 'dx'].values[0]
    if subject['dx'] != 'Typically Developing Children':
        subject['dx'] = 1
    else:
        subject['dx'] = 0


## Image Preprocessing

In [ ]:
transforms = [
    tio.CropOrPad((164, 164, 164)),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.RescaleIntensity((0, 1)),
    tio.RandomAffine(),
    #tio.RandomNoise(),
    # tio.RandomMotion(),
    # tio.RandomBiasField(),

]
transform = tio.Compose(transforms)
subjects_dataset = tio.SubjectsDataset(dataset, transform=transform)


In [ ]:
#Shows the preprocessed images
for i in range(5):
    subject = subjects_dataset[i]
    image = subject['mri']
    print('Subject ID:', subject['id'])
    print('Age:', subject['age'])
    print('Gender:', subject['gender'])
    print('Diagnosis:', subject['dx'])
    image.plot()
    plt.show()
    print(subjects_dataset[i]['mri'].spatial_shape)

## Train/Test Split

In [ ]:
# split the torchio dataset into train/test with a 80/20 ratio
n = len(subjects_dataset)
n_train = int(0.8 * n)
n_val = n - n_train
train_dataset, val_dataset = torch.utils.data.random_split(subjects_dataset, [n_train, n_val])


# Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=3)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=3)

# print the number of batches in the train and validation loaders
print('Number of batches in the train loader: ' + str(len(train_loader)))
print('Number of batches in the validation loader: ' + str(len(val_loader)))


## Pre-Trained Model

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained CNN model for neuorimaging data
model = torchvision.models.video.r3d_18(pretrained=True)

# Modify the first layer to accept 1 channel
model.stem[0] = nn.Conv3d(1, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)

# Adds regularization in all sequential layers
# # for layer in model.modules():
# #     if isinstance(layer, nn.Conv3d):
# #         nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
# #     elif isinstance(layer, nn.BatchNorm3d):
# #         nn.init.constant_(layer.weight, 1)
# #         nn.init.constant_(layer.bias, 0)



# Modify the fully connected layer
model.fc = nn.Linear(512, 2)

# model.fc = nn.Sequential(
#     nn.Linear(512, 256),
#     nn.ReLU(),
#     nn.Dropout(0.4),
#     nn.Linear(256, 2)
# )

# Print the model architecture
print(model)
model = model.to(device)
print(torch.cuda.get_device_name())


## Training

In [ ]:
from torch.cuda.amp import autocast

# LOSS FUNCTION  
criterion = nn.CrossEntropyLoss()
# criterions = nn.BCEWithLogitsLoss()

# OPTIMIZERS
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.AdamW(model.parameters(), lr=0.001)

# SCHEDULER
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.01)
epochs = 5


print('Training Started')

# TRAINING LOOP
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        print('Batch: ' + str(i))
        inputs = batch['mri']['data'].to(device)
        labels = batch['dx'].to(device)
        with autocast():   
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print('Average Loss: ' + str(running_loss / (i + 1)))
        optimizer.zero_grad()    

    print('Epoch: ' + str(epoch + 1) + ', Loss: ' + str(running_loss / len(train_loader)))
    scheduler.step()


## Evaluation

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for i, batch in enumerate(val_loader):
        inputs = batch['mri']['data'].to(device)
        labels = batch['dx'].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %.2f %%' % (100 * correct / total))
print('Loss: ' + str(running_loss / len(val_loader)))

# Get current time
current_time = time.strftime('%Y-%m-%d_%H-%M-%S')
# Save the model with timestamp
torch.save(model.state_dict(), 'model_' + current_time + '.pth')
print('Model saved as model.pth')


In [ ]:
# Save Checkpoint
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}
torch.save(checkpoint, 'checkpoint.pth')

print('Checkpoint saved as checkpoint.pth')